In [1]:
from bettmensch_ai.pipelines import OutputArtifact, InputArtifact
import torch
from bettmensch_ai_examples.gpt_1.src.train import pretrain, GPT1PretrainConfig, GPT1Trainer, TokenizedBookCorpusOpenSplit
from bettmensch_ai_examples.gpt_1.component import get_source_data_split, get_tokenized_data_split_and_tokenizer, pretrain_and_checkpoint

W0224 23:14:02.651000 9404 torch\distributed\elastic\multiprocessing\redirects.py:27] NOTE: Redirects are currently not supported in Windows or MacOs.
c:\Users\scher\Repositories\bettmensch.ai-examples\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pipeline pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x000001AC5DFFED10>
Base Component pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x000001AC5DFFED10>
Base Component pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x000001AC5DFFED10>
Pipeline pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x000001AC5DFFED10>
Base Component pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x000001AC5DFFED10>
Base Component pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x000001AC5DFFED10>
Pipeline pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x000001AC5DFFED10>
Base Component pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x000001AC5DFFED10>
Base Component pipeline context: <bettmensch_ai.pi

In [2]:
get_source_data = False

if get_source_data:
    get_source_data_split(
        data_out=OutputArtifact(name="source_data_train")
    )

In [3]:
preprocess_data = False
N_TOTAL = 74004228
train_split = 0.9
n_train_observations = int(train_split * N_TOTAL)
sequence_length_test = 128

if preprocess_data:
    get_tokenized_data_split_and_tokenizer(
        source_data_split=InputArtifact(name="source_data_train"),
        end_index=n_train_observations,
        sequence_length=sequence_length_test,
        tokenized_data_out=OutputArtifact(name="tokenized_data_train"),
        tokenizer_out=OutputArtifact(name="tokenizer_train")
    )

    get_tokenized_data_split_and_tokenizer(
        source_data_split=InputArtifact(name="source_data_train"),
        start_index=n_train_observations,
        end_index=-1,
        sequence_length=sequence_length_test,
        tokenized_data_out=OutputArtifact(name="tokenized_data_validation"),
        tokenizer_out=OutputArtifact(name="tokenizer_validation")
    )

In [4]:
config_test = GPT1PretrainConfig.from_file("C:\\Users\\scher\\Repositories\\bettmensch.ai-examples\\sdk\\bettmensch_ai_examples\\gpt_1\\src\\config.yaml")
print(config_test.data)
print(config_test.tokenizer)
print(config_test.model)
print(config_test.trainer)

{'train': {'path': 'tokenized_data_validation', 'dataloader': {'batch_size': 45, 'shuffle': True}}, 'validation': {'use': False, 'path': 'tokenized_data_validation', 'dataloader': {'batch_size': 80, 'shuffle': False}}}
{'path': 'tokenizer_train'}
{'architecture': {'n_tokens': 128, 'dim_embed': 768, 'n_decoder_layers': 12, 'n_heads': 12, 'dropout': 0.1}, 'misc': {'verbose': False}}
{'optimizer': {'lr': 0.00025, 'weight_decay': 0.01, 'betas': [0.9, 0.95], 'eps': 0.0001}, 'scheduler': {'linear': {'start_factor': 0.0001, 'end_factor': 1, 'total_iters': 1}, 'cosine': {'T_max': 500, 'eta_min': 0}, 'sequential': {'milestones': [1000]}}, 'training': {'n_epochs': 1, 'n_batches': -1, 'display_step': 10, 'use_amp': False, 'scale_gradients': True}}


In [5]:
train = True

if train:
    pretrain(config_test)

EPOCH 1:


RuntimeError: expected scalar type Float but found Half

In [ ]:
resume_training = False

if resume_training:
    torch.set_default_device("cuda")
    train_data = TokenizedBookCorpusOpenSplit(config_test.data['train']['path'])

    train_loader = torch.utils.data.DataLoader(
        train_data, collate_fn=TokenizedBookCorpusOpenSplit.collate_batch, **config_test.data['train']['dataloader'],generator=torch.Generator(device="cuda")
    )
    torch.set_default_device("cuda")
    trainer = GPT1Trainer(config_test.trainer)
    trainer.train(train_loader=train_loader,checkpoint="2")